# Unicode, UTF-8, and Byte-Pair Encoding (BPE)

This notebook illustrates:
1. How Unicode strings are represented
2. Converting Unicode to UTF-8 byte sequences
3. How byte-level BPE tokenization works
4. Encoding and decoding text using BPE

## 1. Unicode Basics

**Unicode** is a standard that assigns a unique number (called a **code point**) to every character across all languages and symbols.

- Code points are written as `U+XXXX` (hexadecimal)
- Python strings are Unicode by default (Python 3)

In [ ]:
# Unicode characters and their code points
examples = ['A', 'z', '5', ' ', 'é', '中', '🚀', '→']

print("Char | Code Point | Decimal")
print("-" * 30)
for char in examples:
    code_point = ord(char)  # Get Unicode code point
    print(f"  {char}  |   U+{code_point:04X}   |  {code_point}")

## 2. UTF-8 Encoding

**UTF-8** is a variable-length encoding that converts Unicode code points to bytes:

| Code Point Range | Bytes | Byte Pattern |
|-----------------|-------|---------------|
| U+0000 - U+007F | 1 | `0xxxxxxx` |
| U+0080 - U+07FF | 2 | `110xxxxx 10xxxxxx` |
| U+0800 - U+FFFF | 3 | `1110xxxx 10xxxxxx 10xxxxxx` |
| U+10000 - U+10FFFF | 4 | `11110xxx 10xxxxxx 10xxxxxx 10xxxxxx` |

ASCII characters (0-127) use just 1 byte, making UTF-8 backward compatible with ASCII.

In [ ]:
# Converting Unicode strings to UTF-8 bytes
text = "Hello, 世界! 🌍"

# Encode to UTF-8 bytes
utf8_bytes = text.encode('utf-8')

print(f"Original string: {text}")
print(f"String length (characters): {len(text)}")
print(f"UTF-8 bytes: {utf8_bytes}")
print(f"Byte length: {len(utf8_bytes)}")

In [ ]:
# Detailed breakdown of UTF-8 encoding per character
print("Character-by-character UTF-8 breakdown:\n")
print("Char | UTF-8 Bytes (hex) | UTF-8 Bytes (decimal)")
print("-" * 55)

for char in text:
    char_bytes = char.encode('utf-8')
    hex_repr = ' '.join(f'{b:02X}' for b in char_bytes)
    dec_repr = ' '.join(f'{b:3d}' for b in char_bytes)
    display_char = char if char != ' ' else '(space)'
    print(f"  {display_char:4} |  {hex_repr:17} | {dec_repr}")

In [ ]:
# Decoding UTF-8 bytes back to a string
decoded_text = utf8_bytes.decode('utf-8')
print(f"Decoded string: {decoded_text}")
print(f"Roundtrip successful: {text == decoded_text}")

## 3. GPT-2 Style Byte Encoding

GPT-2 (and many modern tokenizers) work at the **byte level**, but they map each byte (0-255) to a printable Unicode character. This avoids issues with non-printable bytes.

The mapping:
- Printable ASCII characters (`!` to `~`, plus space) map to themselves
- Other bytes map to characters starting at `U+0100` (Ā, ā, Ă, etc.)

In [ ]:
def bytes_to_unicode():
    """
    Returns a mapping from bytes (0-255) to unicode characters.
    This is the GPT-2 byte encoding scheme.
    """
    # Printable ASCII ranges that map to themselves
    bs = list(range(ord("!"), ord("~") + 1))  # 33-126
    bs += list(range(ord("¡"), ord("¬") + 1))  # 161-172
    bs += list(range(ord("®"), ord("ÿ") + 1))  # 174-255
    
    cs = bs[:]
    
    # Map remaining bytes (0-32, 127-160, 173) to characters starting at 256
    n = 0
    for b in range(256):
        if b not in bs:
            bs.append(b)
            cs.append(256 + n)
            n += 1
    
    cs = [chr(c) for c in cs]
    return dict(zip(bs, cs))

byte_encoder = bytes_to_unicode()
byte_decoder = {v: k for k, v in byte_encoder.items()}

print("Sample byte-to-character mappings:")
print("\nByte | Char | Notes")
print("-" * 35)
sample_bytes = [0, 10, 32, 65, 97, 127, 128, 200, 255]
for b in sample_bytes:
    char = byte_encoder[b]
    note = ""
    if b == 0: note = "(null byte)"
    elif b == 10: note = "(newline)"
    elif b == 32: note = "(space)"
    elif b == 65: note = "(ASCII 'A')"
    elif b == 97: note = "(ASCII 'a')"
    elif b == 127: note = "(DEL)"
    print(f" {b:3d} |  {char}   | {note}")

In [ ]:
# Encode text to GPT-2 style byte representation
def text_to_gpt2_bytes(text):
    """Convert text to GPT-2's byte representation."""
    utf8_bytes = text.encode('utf-8')
    return ''.join(byte_encoder[b] for b in utf8_bytes)

def gpt2_bytes_to_text(gpt2_str):
    """Convert GPT-2 byte representation back to text."""
    byte_values = bytes([byte_decoder[c] for c in gpt2_str])
    return byte_values.decode('utf-8')

# Example
original = "Hello, 世界! 🌍"
gpt2_repr = text_to_gpt2_bytes(original)
recovered = gpt2_bytes_to_text(gpt2_repr)

print(f"Original:         {original}")
print(f"GPT-2 byte repr:  {gpt2_repr}")
print(f"Recovered:        {recovered}")
print(f"\nNotice how multi-byte UTF-8 characters become multiple GPT-2 tokens")

## 4. Byte-Pair Encoding (BPE)

**BPE** is a compression algorithm adapted for tokenization:

1. Start with a vocabulary of individual bytes (or characters)
2. Find the most frequent adjacent pair of tokens
3. Merge that pair into a new token
4. Repeat until vocabulary reaches desired size

This creates a vocabulary of subword units that balances:
- Common words as single tokens
- Rare words split into meaningful pieces

In [ ]:
from collections import Counter

def get_pair_counts(token_sequences):
    """Count frequency of adjacent token pairs."""
    pairs = Counter()
    for seq in token_sequences:
        for i in range(len(seq) - 1):
            pairs[(seq[i], seq[i + 1])] += 1
    return pairs

def merge_pair(token_sequences, pair, new_token):
    """Merge all occurrences of a pair into a new token."""
    new_sequences = []
    for seq in token_sequences:
        new_seq = []
        i = 0
        while i < len(seq):
            if i < len(seq) - 1 and (seq[i], seq[i + 1]) == pair:
                new_seq.append(new_token)
                i += 2
            else:
                new_seq.append(seq[i])
                i += 1
        new_sequences.append(new_seq)
    return new_sequences

def train_bpe(text, num_merges):
    """Train BPE on text and return vocabulary and merge rules."""
    # Start with individual characters as tokens
    tokens = [list(word) for word in text.split()]
    
    # Initial vocabulary is all unique characters
    vocab = set(char for word in tokens for char in word)
    merges = []
    
    print(f"Initial vocabulary size: {len(vocab)}")
    print(f"Initial tokens: {tokens[:3]}...\n")
    
    for i in range(num_merges):
        pairs = get_pair_counts(tokens)
        if not pairs:
            break
        
        # Find most frequent pair
        best_pair = max(pairs, key=pairs.get)
        new_token = best_pair[0] + best_pair[1]
        
        print(f"Merge {i+1}: '{best_pair[0]}' + '{best_pair[1]}' -> '{new_token}' (count: {pairs[best_pair]})")
        
        # Merge the pair
        tokens = merge_pair(tokens, best_pair, new_token)
        vocab.add(new_token)
        merges.append(best_pair)
    
    print(f"\nFinal vocabulary size: {len(vocab)}")
    return vocab, merges, tokens

# Train BPE on sample text
sample_text = "low lower lowest low lower lowest newer newest wide wider widest"
vocab, merges, final_tokens = train_bpe(sample_text, num_merges=10)

In [ ]:
# Show final tokenization
print("Final tokenization:")
print(final_tokens)

print("\nMerge rules (in order):")
for i, (a, b) in enumerate(merges):
    print(f"  {i+1}. '{a}' + '{b}' -> '{a}{b}'")

## 5. BPE Encoding (Tokenization)

To encode new text with a trained BPE model:
1. Split text into characters (or bytes)
2. Apply merge rules in the order they were learned
3. Map resulting tokens to integer IDs

In [ ]:
def bpe_encode(text, merges):
    """Encode text using learned BPE merges."""
    words = text.split()
    encoded_words = []
    
    for word in words:
        tokens = list(word)  # Start with characters
        
        # Apply merges in order
        for pair in merges:
            i = 0
            new_tokens = []
            while i < len(tokens):
                if i < len(tokens) - 1 and (tokens[i], tokens[i + 1]) == pair:
                    new_tokens.append(tokens[i] + tokens[i + 1])
                    i += 2
                else:
                    new_tokens.append(tokens[i])
                    i += 1
            tokens = new_tokens
        
        encoded_words.append(tokens)
    
    return encoded_words

# Test encoding
test_words = ["low", "lower", "lowest", "new", "newer", "unknown"]
print("BPE tokenization results:\n")
for word in test_words:
    tokens = bpe_encode(word, merges)[0]
    print(f"  '{word}' -> {tokens}")

## 6. BPE Decoding

Decoding is simple: concatenate tokens and convert bytes back to text.

In [ ]:
def bpe_decode(token_sequences):
    """Decode BPE tokens back to text."""
    words = [''.join(tokens) for tokens in token_sequences]
    return ' '.join(words)

# Round-trip test
original_text = "lower newest wider"
encoded = bpe_encode(original_text, merges)
decoded = bpe_decode(encoded)

print(f"Original: '{original_text}'")
print(f"Encoded:  {encoded}")
print(f"Decoded:  '{decoded}'")
print(f"Roundtrip OK: {original_text == decoded}")

## 7. Byte-Level BPE (Putting It All Together)

Modern tokenizers like GPT-2 combine:
1. **UTF-8 encoding** - Convert Unicode text to bytes
2. **Byte-to-character mapping** - Make all bytes printable
3. **BPE** - Learn subword merges on the byte representation

This allows handling ANY text (any language, emoji, binary data) with a fixed vocabulary.

In [ ]:
# Complete byte-level BPE pipeline demonstration
def demonstrate_byte_level_bpe(text):
    print(f"Input text: {text}")
    print(f"Character count: {len(text)}\n")
    
    # Step 1: UTF-8 encode
    utf8_bytes = text.encode('utf-8')
    print(f"Step 1 - UTF-8 bytes: {list(utf8_bytes)}")
    print(f"Byte count: {len(utf8_bytes)}\n")
    
    # Step 2: Map to printable characters (GPT-2 style)
    gpt2_chars = ''.join(byte_encoder[b] for b in utf8_bytes)
    print(f"Step 2 - GPT-2 byte chars: {gpt2_chars}")
    print(f"Char count: {len(gpt2_chars)}\n")
    
    # Step 3: This is where BPE would tokenize
    print("Step 3 - BPE would merge frequent pairs into tokens")
    print("(Using pre-trained merges from vocabulary)\n")
    
    # Decoding reverses the process
    print("Decoding:")
    recovered_bytes = bytes([byte_decoder[c] for c in gpt2_chars])
    recovered_text = recovered_bytes.decode('utf-8')
    print(f"  GPT-2 chars -> bytes -> UTF-8 decode -> '{recovered_text}'")

# Try with various Unicode text
demonstrate_byte_level_bpe("Hello 世界 🎉")

## Summary

| Step | Process | Purpose |
|------|---------|--------|
| 1 | Unicode string | Human-readable text |
| 2 | UTF-8 encoding | Convert to bytes (1-4 bytes per char) |
| 3 | Byte-to-char mapping | Make all 256 byte values printable |
| 4 | BPE tokenization | Compress into subword tokens |
| 5 | Token IDs | Integer indices for the model |

**Key benefits of byte-level BPE:**
- Handles ANY Unicode text without "unknown" tokens
- Efficient vocabulary (common words = single tokens)
- Graceful degradation (rare text = more tokens, but still works)
- Language-agnostic (no language-specific preprocessing)

In [ ]:
# Final interactive example - try your own text!
your_text = "Try any text here: café, naïve, 日本語, emoji 🎭"

print("=" * 60)
print("BYTE-LEVEL ENCODING BREAKDOWN")
print("=" * 60)

for char in your_text[:20]:  # First 20 chars
    utf8 = char.encode('utf-8')
    gpt2 = ''.join(byte_encoder[b] for b in utf8)
    display = char if char != ' ' else '␣'
    print(f"{display:2} -> UTF-8: {list(utf8):20} -> GPT-2: {gpt2}")